Här är koden för den färdiga modellen som skall användas i Streamlit-appen. Jag använder samma transformer och pipeline som i den andra Notebooken med test och utvärdering av modell. Då SVC fick accuracy på 0.978 och högt F1 score så är det den modellen som används nedan.

In [2]:
# ========================================
# TRÄNA FINAL MNIST SVC MODELL
# (samma pipeline som i sluttest-notebooken)
# ========================================

import time
import joblib
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from datetime import timedelta
from sklearn.decomposition import PCA


# Läser in data från OpenML
print("Läser in MNIST data...")
mnist = fetch_openml("mnist_784", version=1, cache=True, as_frame=False)
X = mnist.data
y = mnist.target.astype(int)
print(f"X shape: {X.shape}, y shape: {y.shape}")


# Skapar en transformer för att minska antal pixlar
class ResizeImages(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_images = X.reshape(-1, 28, 28)
        X_cropped = X_images[:, 4:24, 4:24]
        return X_cropped.reshape(-1, 20 * 20)


# Använder samma pipeline som i andra notebooken
model = Pipeline([
    ("resize", ResizeImages()),
    ("scaler", StandardScaler()),
    ("pca", PCA(n_components=0.95)),  # reducerar features men behåller 95% av variansen 
    ("classifier", SVC(kernel="rbf", probability=True))
])


# Tränar om modellen på ALL data, inkl tidigare test, och sparar inför appen
print("Tränar SVC-modell på hela datasetet...")
start = time.time()
model.fit(X, y)
execution_time = time.time() - start
print (f"Tid att träna SVC: {timedelta(seconds=execution_time)}")

joblib.dump(model, "mnist_svc_model.pkl")

print("✅ Modell sparad som 'mnist_svc_model.pkl'")


Läser in MNIST data...
X shape: (70000, 784), y shape: (70000,)
Tränar SVC-modell på hela datasetet...
Tid att träna SVC: 0:11:54.319548
✅ Modell sparad som 'mnist_svc_model.pkl'
